# **FARM**

Set-Up

In [ ]:
#installing FARM
!git clone https://github.com/deepset-ai/FARM.git
!pip install -r FARM/requirements.txt
!pip install FARM/

In [ ]:
#importing modules
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import MultiLabelTextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.infer import Inferencer
from farm.train import Trainer
from farm.utils import MLFlowLogger, initialize_device_settings, set_all_seeds, MLFlowLogger
import logging
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
import numpy as np

In [ ]:
#Tracking the results
#ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
#ml_logger.init_experiment(experiment_name="Empathy", run_name="final")

In [ ]:
#Fetch the right device 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Data Preprocessing


In [ ]:
#Initializing parameters 
set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 3
learning_rate = 3e-5
embeds_dropout_prob = 0.1
batch_size = 8
evaluate_every = 100

lang_model = "bert-base-german-cased"
do_lower_case = False 


In [ ]:
#Loading the dataset
df = pd.read_csv("/content/drive/My Drive/Data/dataset_groupedempathylevel.csv")
df.drop(df[df['length']<=3].index, inplace = True)  #droping all rows that are smaller/equal 3 in length
columns_to_keep = ['text','classID', 'f_4', 'f_5'] #dropping the rest
df = df[columns_to_keep]
df


In [ ]:
#Splitting the dataset to train/test
from numpy.random import RandomState
rng = RandomState()
components_train = df.sample(frac=0.8, random_state=42)
components_test = df.loc[~df.index.isin(components_train.index)]
components_train.to_csv('/content/drive/My Drive/Data/Farm/train.tsv', sep='\t', index=False, header=True)
components_test.to_csv('/content/drive/My Drive/Data/Farm/test.tsv', sep='\t', index=False, header=True)

In [ ]:
#Creating a tokenizer (here: BERT tokenizer loaded with german model)
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

# Model Components

In [ ]:
#Creating a processor to handle conversion from raw text to PyTorch Dataset
label_list = ['strength', "weakness", "suggestions", "None"] #labels in the data set
metric = "f1_macro"  # desired metric for evaluation

processor = TextClassificationProcessor(tokenizer=tokenizer,
                                            max_seq_len=512, # BERT can only handle sequence lengths of up to 512
                                            data_dir='/content/drive/My Drive/Data/Farm', 
                                            label_list=label_list,
                                            label_column_name="classID", 
                                            metric=metric,
                                            quote_char='"',
                                            multilabel=True,
                                            train_filename="train.tsv",
                                            dev_filename=None,
                                            test_filename="test.tsv",
                                            dev_split=0.2 # this will extract 20% of the train set to create a dev set
                                          )

In [ ]:
#Creating a DataSilo to load various datasets(train/test/dev)
data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

In [ ]:
#Loading the pretrained BERT german model
language_model = LanguageModel.load(lang_model)

#Define a prediction head that fits for text classification with multiple labels
prediction_head = MultiLabelTextClassificationHead(class_weights=data_silo.calculate_class_weights(task_name="text_classification"),num_labels=len(label_list))

#Create the model
model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[prediction_head],
        embeds_dropout_prob=embeds_dropout_prob,
        lm_output_types=["per_sequence"],
        device=device)
model.fit_heads_to_lm()

In [ ]:
#Creating the optimizer
model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=learning_rate,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=n_epochs)

In [ ]:
#Feeding to the Trainer
trainer = Trainer(
        model=model,
        optimizer=optimizer,
        data_silo=data_silo,
        epochs=n_epochs,
        n_gpu=n_gpu,
        lr_schedule=lr_schedule,
        evaluate_every=evaluate_every,
        device=device)
#Training and growing
model = trainer.train()

In [ ]:
#Save the model
save_dir_components = "/content/drive/My Drive/Data/Farm/saved_models/components_final"
model.save(save_dir_components)
processor.save(save_dir_components)

# **Model f_4 (emotional empathy)**

In [ ]:
#Creating a processor to handle conversion from raw text to PyTorch Dataset
label_list = ['non-empathic', 'empathic', "neutral", "None"] #labels in the data set
metric = "f1_macro"  # desired metric for evaluation

processor = TextClassificationProcessor(tokenizer=tokenizer,
                                            max_seq_len=512, # BERT can only handle sequence lengths of up to 512
                                            data_dir='/content/drive/My Drive/Data/Farm', 
                                            label_list=label_list,
                                            label_column_name="f_4", 
                                            metric=metric,
                                            quote_char='"',
                                            multilabel=True,
                                            train_filename="train.tsv",
                                            dev_filename=None,
                                            test_filename="test.tsv",
                                            dev_split=0.2 # this will extract 20% of the train set to create a dev set
                                          )

In [ ]:
#Creating a DataSilo to load various datasets(train/test/dev)
data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

In [ ]:
#Loading the pretrained BERT german model
language_model = LanguageModel.load(lang_model)

#Define a prediction head that fits for text classification with multiple labels
prediction_head = MultiLabelTextClassificationHead(class_weights=data_silo.calculate_class_weights(task_name="text_classification"),num_labels=len(label_list))

#Create the model
model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[prediction_head],
        embeds_dropout_prob=embeds_dropout_prob,
        lm_output_types=["per_sequence"],
        device=device)
model.fit_heads_to_lm()

In [ ]:
#Creating the optimizer
model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=learning_rate,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=n_epochs)

In [ ]:
#Feeding to the Trainer
trainer = Trainer(
        model=model,
        optimizer=optimizer,
        data_silo=data_silo,
        epochs=n_epochs,
        n_gpu=n_gpu,
        lr_schedule=lr_schedule,
        evaluate_every=evaluate_every,
        device=device)
#Training and growing
model = trainer.train()

In [ ]:
#Save the model
save_dir_f4 = "/content/drive/My Drive/Data/Farm/saved_models/emotionalempathy_final"
model.save(save_dir_f4)
processor.save(save_dir_f4)

# **Model f_5 (cognitive empathy)**

In [ ]:
#Creating a processor to handle conversion from raw text to PyTorch Dataset
label_list = ['non-empathic', 'empathic', "neutral", "None"] #labels in the data set
metric = "f1_macro"  # desired metric for evaluation

processor = TextClassificationProcessor(tokenizer=tokenizer,
                                            max_seq_len=512, # BERT can only handle sequence lengths of up to 512
                                            data_dir='/content/drive/My Drive/Data/Farm', 
                                            label_list=label_list,
                                            label_column_name="f_5", 
                                            metric=metric,
                                            quote_char='"',
                                            multilabel=True,
                                            train_filename="train.tsv",
                                            dev_filename=None,
                                            test_filename="test.tsv",
                                            dev_split=0.2 # this will extract 20% of the train set to create a dev set
                                          )

In [ ]:
#Creating a DataSilo to load various datasets(train/test/dev)
data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

In [ ]:
#Loading the pretrained BERT german model
language_model = LanguageModel.load(lang_model)

#Define a prediction head that fits for text classification with multiple labels
prediction_head = MultiLabelTextClassificationHead(class_weights=data_silo.calculate_class_weights(task_name="text_classification"), num_labels=len(label_list))

#Create the model
model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[prediction_head],
        embeds_dropout_prob=embeds_dropout_prob,
        lm_output_types=["per_sequence"],
        device=device)
model.fit_heads_to_lm()


In [ ]:
#Creating the optimizer
model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=learning_rate,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=n_epochs)

In [ ]:
#Feeding to the Trainer
trainer = Trainer(
        model=model,
        optimizer=optimizer,
        data_silo=data_silo,
        epochs=n_epochs,
        n_gpu=n_gpu,
        lr_schedule=lr_schedule,
        evaluate_every=evaluate_every,
        device=device)
#Training and growing
model = trainer.train()

In [ ]:
#Save the model
save_dir_f5 = "/content/drive/My Drive/Data/Farm/saved_models/cognitiveempathy_final"
model.save(save_dir_f5)
processor.save(save_dir_f5)

# Test on Sample

In [ ]:
#Test the model on a sample 
from farm.infer import Inferencer
from pprint import PrettyPrinter


basic_texts = [{"text": "Das Template wurde gut umgesetzt. Die Darstellung ist schlüssig, Persona und User Cycle passen zusammen."
}]
#inferenced_model = Inferencer.load("/content/drive/My Drive/Data/Farm/saved_models/components_final")
inferenced_model= Inferencer.load("/content/drive/My Drive/Data/Farm/saved_models/cognitiveempathy_final")
#inferenced_model= Inferencer.load("/content/drive/My Drive/Data/Farm/saved_models/emotionalempathy_final")
result = inferenced_model.inference_from_dicts(dicts=basic_texts)
PrettyPrinter().pprint(result)   

Codes based on https://deepset.ai/german-bert